In [1]:
# Generate audio data for the beamformer. This will not be used when the algoritm runs with real data
# This program takes around 5 minutes to generate audio data (with one source) for 1 second.

import numpy as np

import config
from common import Array


In [2]:
# GLOBAL VARIABLES
c = 340                     # propagation speed of sound
# Initialization
# f_sampling, t_start, t_end, away_distance are user defined variables
f_sampling = 16000  # config.f_sampling                      # sampling frequency in Hz
t_start = config.t_start                            # start time of simulation
t_end = config.t_end                                # end time of simulation
t_total = t_end - t_start                           # total simulation time
t = np.linspace(t_start, t_end, t_total*f_sampling)  # time vector

# distance between the array and sources
away_distance = config.away_distance

filename = "array_audio_signals3.npy"


In [3]:
class Source:
    """Audio source"""

    def __init__(self, f_start, f_end, f_res, theta_deg, phi_deg, rho, t_start, t_end):
        self.theta = theta_deg*np.pi/180
        self.phi = phi_deg*np.pi/180
        self.frequency = np.linspace(f_start, f_end, f_res)
        self.t_start = t_start
        self.t_end = t_end
        self.rho = rho


In [4]:
def r_vec(theta, phi):
    return np.array([(np.sin(theta)*np.cos(phi)),
                     np.sin(theta)*np.sin(phi), np.cos(theta)])


In [5]:
def generate_array_signals(matrix_array, sources, t):
    r_prime = matrix_array.r_prime
    Audio_signal = np.zeros((len(t), len(r_prime[0, :])))

    for sample in range(len(t)):
        # print stuff so user know how many samples that have been generated
        if (sample+1 in np.linspace(0, len(t), 11)) or (sample == 0):
            # print stuff so user know how many samples that have been generated
            print(sample+1)
        for mic in range(len(r_prime[0, :])):
            x_i = r_prime[0, mic]
            y_i = r_prime[1, mic]
            temp_signal_sample = 0
            for source in sources:
                if (source.t_start < t[sample]) and (t[sample] < source.t_end):
                    frequencies_ps = source.frequency
                    theta_source = source.theta
                    phi_source = source.phi
                    rho_source = source.rho

                    for freq in frequencies_ps:
                        k = 2*np.pi*freq/c
                        r_1 = np.array([x_i, y_i, 0])
                        r_2 = rho_source * r_vec(theta_source, phi_source)
                        norm_coeff = np.linalg.norm(r_2-r_1)
                        phase_offset = -k*norm_coeff
                        element_amplitude = 1/norm_coeff
                        temp_signal_sample += element_amplitude * \
                            np.sin(2*np.pi * freq * t[sample] + phase_offset)

            Audio_signal[sample, mic] = temp_signal_sample
    return Audio_signal


In [6]:
# set up arrays
arrays = [Array(config.r_a1, config.distance, config.rows, config.columns)]

array_matrices = arrays
sub_arrays = len(array_matrices)


In [7]:
# Create and place out sources
# source1 and source2 below can be generated in parallell
source1 = Source(config.f_start1, config.f_end1, config.f_res1,
                 config.theta_deg1, config.phi_deg1, config.away_distance, config.t_start1, config.t_end1)
source2 = Source(config.f_start2, config.f_end2, config.f_res2,
                 config.theta_deg2, config.phi_deg2, config.away_distance, config.t_start2, config.t_end2)
sources = np.array([source1, source2])

 # GENERATE AUDIO SIGNAL
 
array_audio_signals = np.zeros((sub_arrays), dtype=object)
print('Number of samples generated (of '+str(f_sampling*t_total)+'):')
for array in range(sub_arrays):  # PARALLELL
    # generate the audio signals on each array-element for each sub-array
    temp_signal = generate_array_signals(
        array_matrices[array], sources, t)  # t = time
    array_audio_signals[array] = temp_signal
    print('Audio signal for array '+str(array+1)+' generated')

np.save(filename, array_audio_signals)


Number of samples generated (of 16000):
1
1600
3200
4800
6400
8000
9600
11200
12800
14400
16000
Audio signal for array 1 generated
